<a href="https://colab.research.google.com/github/shpotes/tensorflowers/blob/resnet-baseline/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/
!git clone https://github.com/shpotes/tensorflowers.git
%cd tensorflowers
!pip install -q -r requirements.txt

In [ ]:
import torch.nn as nn
from torchvision import models
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

from src.dataset import TFColDataModule
from src.models.pl_module import HydraModule

2021-11-14 15:14:34.581200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 15:14:34.581216: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
dm = TFColDataModule(
    image_transforms=transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
)

In [ ]:
base_backbone = models.resnet18(pretrained=True)
backbone = nn.Sequential(*list(base_backbone.children())[:-1])
model = HydraModule(
    backbone,
    lr=3e-4,
)

In [ ]:
!wandb --login

In [ ]:
logger = WandbLogger(
    project="challenge", 
    name="baseline",
    entity="tensorflowers",
)
trainer = pl.Trainer(
    max_epochs=10,
    gpus=1,
    logger=logger    
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, dm)